<a href="https://colab.research.google.com/github/dh610/Advanced-AI/blob/main/AirNet_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

#!pip install colab_ssh --upgrade

#from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
#launch_ssh_cloudflared(password="1234")

Mounted at /content/drive


In [13]:
import sys
import os

sys.path.append("/content/drive/MyDrive/Advanced-AI/AirNet/")
os.chdir("/content/drive/MyDrive/Advanced-AI/AirNet/")
derain_path = "/content/drive/MyDrive/Advanced-AI/InstructIR/test-data/Rain100L/"
dehaze_path = "/content/drive/MyDrive/Advanced-AI/InstructIR/test-data/SOTS/"
denoise_path = "/content/drive/MyDrive/Advanced-AI/InstructIR/test-data/denoising_testsets/denoise"
!ls

airnet.yaml  cloudflared      data	demo.py  option.py    README.md  test.py   utils
ckpt	     cloudflared.log  data_dir	net	 __pycache__  test	 train.py


In [9]:
!python train.py --denoise_dir="$denoise_path" --derain_dir="$derain_path" --dehaze_dir="$dehaze_path"

Start training...
  0% 0/400 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/content/drive/MyDrive/Advanced-AI/AirNet/train.py", line 34, in <module>
    for ([clean_name, de_id], degrad_patch_1, degrad_patch_2, clean_patch_1, clean_patch_2) in tqdm(trainloader):
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 630, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 673, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] f

In [16]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import yaml
import random
import torch

from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch
import torch.optim as optim
import torch.nn as nn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
data_dir = pwd + "RainTrainL/"

denoise_input_dir = data_dir + "gaussian_sigma_5/"
derain_input_dir = data_dir + "rain/"
target_dir = data_dir + "target/"


In [18]:
class ImagePairDataset(Dataset):
    def __init__(self, input_dir, target_dir, transform=None):
        self.input_dir = input_dir
        self.target_dir = target_dir
        self.transform = transform
        self.image_filenames = [f for f in os.listdir(input_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        input_image_path = os.path.join(self.input_dir, self.image_filenames[idx])
        input_image = Image.open(input_image_path).convert("RGB")

        target_image_path = os.path.join(self.target_dir, self.image_filenames[idx])
        target_image = Image.open(target_image_path).convert("RGB")

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

denoise_dataset = ImagePairDataset(input_dir=denoise_input_dir, target_dir=target_dir, transform=transform)
derain_dataset = ImagePairDataset(input_dir=derain_input_dir, target_dir=target_dir, transform=transform)

dataset = denoise_dataset + derain_dataset

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)




In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from runpy import run_path
import os
from torch.optim.lr_scheduler import CosineAnnealingLR
from net.model import AirNet
from option import options as opt

opt.cuda = 1
model = AirNet(opt)
model = model.cuda()

optimizer = optim.Adam(model.parameters(), lr=opt.lr)
CE = nn.CrossEntropyLoss().cuda()
l1 = nn.L1Loss().cuda()
#scheduler = CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-6)

In [28]:
opt

Namespace(cuda=1, epochs=10, epochs_encoder=100, lr=0.001, de_type=['denoise_15', 'denoise_25', 'denoise_50', 'derain', 'dehaze'], patch_size=128, encoder_dim=256, num_workers=0, data_file_dir='data_dir/', denoise_dir='data/Train/Denoise/', derain_dir='data/Train/Derain/', dehaze_dir='data/Train/Dehaze/', output_path='output/', ckpt_path='ckpt/Denoise/', batch_size=5)

In [ ]:
from tqdm import tqdm

num_epochs = 5
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in tqdm(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()

        optimizer.zero_grad()

        outputs = model(inputs)

        if epoch < opt.epochs_encoder:
            _, output, target, _ = model.E(x_query=degrad_patch_1, x_key=degrad_patch_2)
            contrast_loss = CE(output, target)
            loss = contrast_loss
        else:
            restored, output, target = model(x_query=degrad_patch_1, x_key=degrad_patch_2)
            contrast_loss = CE(output, target)
            l1_loss = l1(restored, clean_patch_1)
            loss = l1_loss + 0.1 * contrast_loss

        loss.backward()
        optimizer.step()

        running_loss += loss.item()



  0%|          | 0/90 [00:08<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 42.81 MiB is free. Process 33388 has 39.51 GiB memory in use. Of the allocated memory 38.93 GiB is allocated by PyTorch, and 91.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

model.eval()
val_loss = 0.0
with torch.no_grad():
    for data in test_loader:
        inputs = data['input'].cuda()
        targets = data['target'].cuda()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        val_loss += loss.item()
val_loss /= len(test_loader)
print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}')